## Diet Problem

The diet problem aims at finding the number of servings of various foods to buy to minimize the total cost. The minimum-cost diet plan should satisfy specific nutritional requirements.

### **Data & Model Separation**

While implementing optimization models, the best practice is to have a clean separation between the model and the accompanying data. This allows us to solve the same model as is for different data.

Let us first look at the data for the diet problem. We need to know:

* Foods
* Cost of foods
* Nutritional values per unit of different foods
* Nutritional requirements of the diet plan

    To represent that data, we use the global Gurobi function [multidict](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/func_global.html#multidict) that simplifies dictionary initialization. The multidict function allows to initialise one or more dictionaries in a single statement. The function take a dictionary as its argument, where the value associated with each key is a list of length n. The function splits these lists into individual entries, creating n separate dictionaries. The function returns a list. The first item in this list is the Gurobi [tuplelist](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/tuplelist.html#tuplelist) of shared key values, followed by the n individual Gurobi [tupledict](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/tupledict.html#tupledict) dictionaries.

    The code is available in the [Github Link](https://github.com/K-Molloy/nrdf-optimisation/blob/c97a7fda1cf452df32c0e4c3020a99acd9217081/python/diet2.py).
    
    The youtube link of the video for this Jupyter page is [Youtube](https://www.youtube.com/watch?v=w_WkW1kd5Hc)

### **Data**

In [2]:
import gurobipy as gp
from gurobipy import GRB

categories, minNutrition, maxNutrition = gp.multidict({
  'calories': [1800, 2200],
  'protein': [91, GRB.INFINITY],
  'fat': [0, 65],
    'sodium': [0, 1779] })
foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken':   2.89,
    'hot dog':   1.50,
    'fries':     1.89,
    'macaroni':  2.09,
    'pizza':     1.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.59})

# Nutrition values for the foods
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 320,
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      12,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      10,
    ('ice cream', 'sodium'):   180}

### Model

To implement the model , we are using the following methods:
 - tupledict.prod(coeff.pattern): Returns a linear expression that contains one term for each tuple that is present in both the tupledict and coeff argument; coeff should be python dict that maps tuples to coefficient values
 - Ranged Linear Constraint: An expression of the form LinExpr = [Const1, Const2], where Const1 and Const2 are constraints and LineExpr is a LinExpr object

In [12]:
def solve(catgories, minNutrition,maxNutrition,foods, cost, nutritionValues):
    # Create a new model
    model = gp.Model("diet_plan")

    # Create decision variables on the foods to buy
    buy = model.addVars(foods, name="buy") #Returns a gurobipy tupledict with the foods as keys and the decision variables as values

    # The objective is tom minimize the total cost
    model.setObjective(buy.prod(cost), GRB.MINIMIZE)
    # Nutrition constraints
    model.addConstrs((gp.quicksum(nutritionValues[f,c] * buy[f] for f in foods) 
                      == [minNutrition[c],maxNutrition[c]] for c in categories), name="nutrtition_requirements")
    def printSolution():
        if model.status == GRB.OPTIMAL:
            nutrition = {}
            print(f'\nCost: {round(model.ObjVal,4)}')
            print('\nBuy:')
            for f in foods:
                print(f'\t{f}: {round(buy[f].X,4)}')
            print('\nNutritional Values:')
            for c in categories:
                nutrition[c] = sum(buy[f].X * nutritionValues[f,c] for f in foods)
                print(f'\t{c}: {nutrition[c]}')
        else:
            print('No solution found')

    # Solve the model
    model.optimize()
    printSolution()
    

### Solve the model given the data

In [13]:
solve(categories, minNutrition, maxNutrition, foods, cost, nutritionValues)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-4210U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x33ddb849
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.182886111e+01

Cost: 11.8289

Buy:
	hamburger: 0.6045
	chicken: 0.0
	hot dog: 0.0
	fries: 0.0
	macaroni: 0.0
	pizza: 0.0
	salad: 0.0


### Change data

In [14]:
categories, minNutrition, maxNutrition = gp.multidict({
  'calories': [1500, 1800],
  'protein': [100, GRB.INFINITY],
  'fat': [0, 65],
    'sodium': [0, 1779] })
solve(categories, minNutrition, maxNutrition, foods, cost, nutritionValues)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-4210U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x1acfb671
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.03s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.468750e+02   0.000000e+00      0s
       5    1.1640429e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.164042857e+01

Cost: 11.6404

Buy:
	hamburger: 0.4768
	chicken: 0.0
	hot dog: 0.0
	fries: 0.0
	macaroni: 0.0
	pizza: 0.0
	salad: 0.0
